In [ ]:
from protein import Protein

In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv
import networkx as nx
from holoviews import opts

hv.extension('bokeh')

defaults = dict(width=400, height=400)
hv.opts.defaults(opts.EdgePaths(**defaults), opts.Graph(**defaults), opts.Nodes(**defaults))

In [ ]:
N = 8
node_indices = np.arange(N, dtype=np.int32)
source = np.zeros(N, dtype=np.int32)
target = node_indices

print(source)
print(target)

simple_graph = hv.Graph(((source, target),))
simple_graph.opts(inspection_policy='edges')

In [1]:
from ipywidgets import widgets
from IPython.display import display, clear_output
from protein import *
from link import *
import graph

graph_title=widgets.Text(description="Title:")

def printL(l):
    print("[", end="")
    for e in l:
        print(str(e), end=", ")
    print("]")
    
mygraph=graph.GraphLib()

choice_w=widgets.Dropdown(options=[
    ("Add protein", 1), 
    ("Add link", 2), 
    ("Remove protein", 3), 
    ("Remove link", 4),
    ("List proteins", 5),
    ("List links", 6),
    ("List proteins and links", 7),
    ("Save graph as image", 8),
    ("Export graph to file", 9),
    ("Recover from file", 10)], value=1)
text_p_name=widgets.Text(description="Name: ")
text_description=widgets.Text(description="Description: ")
color_w=widgets.ColorPicker(concise=False, value="black", description="Color: ")
text_p1_link=widgets.Dropdown(options=sorted(mygraph.proteins), description="From: ")
text_p2_link=widgets.Dropdown(options=sorted(mygraph.proteins), description="To: ")
dropdown_proteins=widgets.Dropdown(options=sorted(mygraph.proteins), description="Name: ")
button_w=widgets.Button(icon="check", description="Save")
button_remove_w=widgets.Button(icon="times", description="Remove")
widgetsL=[choice_w, text_p_name, text_description, color_w, text_p1_link, text_p2_link, dropdown_proteins, button_w, button_remove_w]

#Function called when the value of choose_w is changed
def on_change(change):
    if change['name']=='value':
        choose_action()

#Function called when the button is pressed to add link or protein
def on_click_warning(click):
    if choice_w.value==3:
        rm_p=find_protein(mygraph.proteins, dropdown_proteins.value)
        if rm_p not in proteins:
            print("\033[91mProtein "+str(dropdown_proteins.value)+" not in proteins list.\033[0m")
            choose_action()
        else:
            print("Are you sure you want to delete protein "+str(dropdown_proteins.value)+"?")
            link_list=find_links(links, rm_p)
            if len(link_list)!=0:
                print("Removing it will cause also the removal of the following links:")
                for l in link_list:
                    print(str(l))
    elif choice_w.value==4:
        rm_p1=find_protein(mygraph.proteins, text_p1_link.value)
        rm_p2=find_protein(mygraph.proteins, text_p2_link.value)
        rm_l=Link(rm_p1, rm_p2)
        if rm_l not in links:
            print("\033[91mLink from "+str(text_p1_link.value)+" to "+str(text_p2_link.value)+" not in links list.\033[0m")
            choose_action()
        else:
            print("Are you sure you want to delete the link between "+str(text_p1_link.value)+" and "+str(text_p2_link.value)+"?")
    items = [widgets.Button(description="Yes"), widgets.Button(description="No")]
    items[0].on_click(yes_remove_click)
    items[1].on_click(no_remove_click)
    display(widgets.Box(items))  
        
#Function called when the button is pressed to add link or protein
def on_click(click):
    if choice_w.value==1:
        new_p=Protein(_id=len(mygraph.proteins), _name=text_p_name.value, _color=color_w.value, _description=text_description.value)
        (nxG, pL)=mygraph.add_protein(new_p)
        if nxG==None and pL==None:
            print("\033[91mProtein "+new_p.name+" already in list.\033[0m")
        else:
            print("\033[94mProtein "+new_p.name+" added.\033[0m")
        choose_Action()
    if choice_w.value==2:
        P1=find_protein(proteins, text_p1_link.value)
        P2=find_protein(proteins, text_p2_link.value)
        if P1==None or P2==None:
            if P1==None:
                print("\033[91mProtein "+text_p1_link.value+" does not exist, cannot create link from "+text_p1_link.value+" to "+text_p2_link.value+".\033[0m")
            if P2==None:
                print("\033[91mProtein "+text_p2_link.value+" does not exist, cannot create link from "+text_p1_link.value+" to "+text_p2_link.value+".\033[0m")
        else:
            new_l=Link(_p1=P1, _p2=P2, _description=text_description.value, _color=color_w.value)
            if new_l not in links:
                links.append(new_l)
                print("\033[94mLink from "+str(new_l.p1)+" to "+str(new_l.p2)+" added.\033[0m")
            else:
                print("\033[91mLink from "+str(new_l.p1)+" to "+str(new_l.p2)+" already in list.\033[0m")
        choose_action()
    if choice_w.value==3:
        pass

def yes_remove_click(click):
    global mygraph
    if choice_w.value==3:
        graph.remove_links(graph.find_links(graph.find_protein(dropdown_proteins.value)))
        graph.remove_protein(dropdown_proteins.value)
    elif choice_w.value==4:
        graph.remove_link_with_ps(text_p1_link.value, text_p2_link.value)
        
def no_remove_click(click):
    choose_action()
    
choice_w.observe(on_change)
button_w.on_click(on_click)
button_remove_w.on_click(on_click_warning)

def choose_action(clear=True):
    global mygraph
    mygraph.write_graph()
    if clear:
        clear_output()
    print("Using graph: "+mygraph.name)
    print("Choose what to do: ")
    display(choice_w)
    choice=choice_w.value
    if choice in [1, 2]: #Add protein or link
        if choice==1:
            print("Insert the data for the new protein:")
            display(text_p_name)
        else:
            print("Insert the data for the new link:")
            text_p1_link.options=sorted(mygraph.proteins)
            text_p2_link.options=sorted(mygraph.proteins)
            display(text_p1_link)
            display(text_p2_link)
        display(color_w)
        display(text_description)
        display(button_w)
    elif choice in [3, 4]: #Remove protein or link
        if choice==3:
            print("Insert the name of the protein to remove:")
            display(dropdown_proteins)            
        else:
            print("Insert the name of the proteins ad the edges of the link:")
            text_p1_link.options=sorted(mygraph.proteins)
            text_p2_link.options=sorted(mygraph.proteins)
            display(text_p1_link)
            display(text_p2_link)
        display(button_remove_w)
    elif choice in [5, 6, 7]: #List elements
        if choice==5: #List the proteins with their links
            if len(mygraph.proteins)==0:
                print([])
            for p in mygraph.proteins:
                print(str(p))
                link_list=graph.find_links(graph.find_protein(p))
                for l in link_list:
                    print("\t"+str(l))
                print()
        elif choice==6: #List the links
            printL(mygraph.links)
        elif choice==7: #List the proteins and the links
            proteins_s=""
            for p in mygraph.proteins:
                proteins_s+=str(p)+"\n"
            links_s=""
            for l in mygraph.links:
                links_s+=str(l)+"\n"
            display(widgets.Box([widgets.Textarea(value=proteins_s), widgets.Textarea(value=links_s)]))     
    
    
    

In [2]:
import networkx as nx
G=nx.Graph()

In [3]:
from ipywidgets import widgets
from IPython.display import display, clear_output
from os import listdir
from time import sleep
from os import write

init_choice_w=widgets.Dropdown(options=[
    ("Create new graph", 1),
    ("Start from last graph", 2),
    ("Open graph", 3),
    ], value=2)
next_button_w=widgets.Button(description="Next", icon="check")
file_chooser_w=widgets.FileUpload()
error=""

def on_change_init(change):
    global error
    clear_output(wait=True)
    if error!="":
        print(error)
    display(init_choice_w)
    if init_choice_w.value==1:
        display(graph_title)
    elif init_choice_w.value==2:
        pass
    elif init_choice_w.value==3:
        display(file_chooser_w)
    display(next_button_w)
        
def on_click_next(click):
    global mygraph 
    global error
    graph_name=""
    succed=True
    if init_choice_w.value==1: #Create new graph
        graph_name=graph_title.value
        file_list=listdir("saved")
        if graph_name in file_list or (graph_name+".graphml") in file_list:
            error="\033[91mFile "+graph_name+" already present, please choose another name or open the file manually.\033[0m"
            succed=False
        else:
            mygraph=graph.Graph(graph_name)
    elif init_choice_w.value==2: #Open last graph
        try:
            with open(".last") as f:
                graph_name=f.readline()
                mygraph.open_graph(graph_name)
        except:
            error="\033[91mNo last file present, please insert new name or manually choose file to open.\033[0m"
            succed=False
    elif init_choice_w.value==3: #Open chosen graph
        graph_name=""
        i=0
        for key in file_chooser_w.value: #Widget is broken so I have to so this loop..........
            if i==0:
                graph_name=key
                i+=1
        mygraph.open_graph(graph_name)
    if succed:
        with open(".last", "w+") as f:
            f.write(graph_name)
        choose_action()
    else:
        first_menu()
    
init_choice_w.observe(on_change_init)
next_button_w.on_click(on_click_next)

def first_menu():
    on_change_init(None)
    



In [4]:
first_menu()

Using graph: nuovografo1.graphml
Choose what to do: 


Dropdown(options=(('Add protein', 1), ('Add link', 2), ('Remove protein', 3), ('Remove link', 4), ('List prote…

Insert the data for the new protein:


Text(value='', description='Name: ')

ColorPicker(value='black', description='Color: ')

Text(value='', description='Description: ')

Button(description='Save', icon='check', style=ButtonStyle())

AttributeError: 'GraphLib' object has no attribute 'create_nxGgraph'

In [ ]:
GD=display(display_id="graph")
GD.display(None)

In [ ]:
G.add_node(1)
G.add_node(2)
G.add_node(3)
G.add_node(4)

In [ ]:
G.add_edge(1,2)

In [ ]:
write_graph(G, G.graph["title"])

In [ ]:
G=nx.Graph()

In [ ]:
G=open_graph("prova1")

In [ ]:
for n in G.nodes:
    print(type(n))
G.nodes

In [ ]:
G1=nx.Graph()
G1.add_node(1, Description="Ciao bellissima proteina")
for i in G1.nodes:
    print(G1.nodes[i]['Description'])
